In [1]:
import pandas as pd
import numpy as np

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Input, Sequential
from keras.layers import LSTM, Embedding, merge, Dense, TimeDistributed, Dropout
from keras.layers import Bidirectional, concatenate, SpatialDropout1D
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from numpy import dot
from numpy.linalg import norm
from numpy import dot


import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from keras.layers import add

Using TensorFlow backend.


In [2]:
R1 = ["Name","Address","Contact","Sex","Guardian's-Name"]
R2 = ["name","Adres","Phone-No","Gender","Father's-Name"]

In [3]:
data = pd.read_csv("ner_dataset.csv", encoding="latin1")

data = data.fillna(method="ffill")
data.tail(5)

,Sentence #,Word,POS,Tag
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O
1048574,Sentence: 47959,attack,NN,O


In [4]:
wordwa = list(set(data["Word"].values))
words = []

for w in wordwa :
    w = str(w)
    w = w.lower()
    words.append(w)
print("Words[0:10] : ")
print(words[0:10])
n_words = len(words); 
n_words

chars = set([w_i for w in words for w_i in w])
n_chars = len(chars)
print("Number of characters : " + str(n_chars))

char2idx = {c: i + 2 for i, c in enumerate(chars)}
char2idx["UNK"] = 1
char2idx["PAD"] = 0

Words[0:10] : 
['latina', 'hats', 'pager', 'acapulco', 'suffered', 'anti-aircraft', 'mengal', '40-year-old', 're-establishing', 'santuario']
Number of characters : 72


In [5]:
max_word_length = 25
embedded_char_vector_length = 74 # Length of character dictionary
char_feature_output = 10

X_char = []
for word in words :
    l1 = []
    #if (len(word) > max_word_length) :
    #    print(len(word))
    for i in range(max_word_length):
        try:
            l1.append(char2idx.get(word[i]))
        except:
            l1.append(char2idx.get("PAD"))    
    
    X_char.append(l1)

In [6]:
char_input = Input(shape=(max_word_length,), dtype='float32', name='char_input')
char_input1 = Embedding(embedded_char_vector_length, 16, input_length = max_word_length)(char_input)
char_input2 = Dropout(0.2)(char_input1)
'''lstm_out_forward = LSTM(char_feature_output, dropout=0.2, recurrent_dropout=0.2)(char_input2)
lstm_out_backward = LSTM(char_feature_output, go_backwards=True, dropout=0.2, recurrent_dropout=0.2)(char_input2)'''

merged = (Bidirectional(LSTM(units=5, return_sequences=False,trainable = False,
                                recurrent_dropout=0.5)))(char_input2)
#merged = merge([lstm_out_forward, lstm_out_backward], mode='concat', concat_axis=1)

#merged = add([lstm_out_forward, lstm_out_backward])
#model = Model(input=[char_input], output=[merged])
model = Model(char_input, merged)


model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
char_input (InputLayer)      (None, 25)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 25, 16)            1184      
_________________________________________________________________
dropout_1 (Dropout)          (None, 25, 16)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 10)                880       
Total params: 2,064
Trainable params: 2,064
Non-trainable params: 0
_________________________________________________________________


In [7]:
X1 = np.array(X_char[0:5])
print(X1)
train_char_vectors = model.predict(X1)


print("\n")
print(train_char_vectors.shape)
print("\n")
print(train_char_vectors)

[[42 31 56 46 59 31  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0]
 [41 31 56 17  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0]
 [33 31 70 29 38  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0]
 [31 68 31 33 37 42 68  5  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0]
 [17 37 27 27 29 38 29 44  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0]]


(5, 10)


[[ 0.00735591 -0.00524074  0.0016058   0.0223674   0.02140207 -0.00288137
   0.03744206  0.00388234  0.00684869 -0.00088662]
 [ 0.00735201 -0.0052412   0.00163848  0.02234489  0.02140816 -0.00311222
   0.01147949 -0.00541078  0.00227256 -0.00565447]
 [ 0.00742292 -0.0052194   0.00169454  0.02244501  0.02157685  0.00806455
   0.02291041  0.01125564  0.00720868 -0.00309351]
 [ 0.00726558 -0.0052624   0.0014167   0.02228511  0.02115925  0.00276302
   0.03926675  0.01618589 -0.00622101  0.00833541]
 [ 0.00721055 -0.00529664  0.00145733  0.02219378  0.02099537  0.00344224
   0.00030344

In [8]:
def pred_new_num_enc(s) :
    
    pred = s

    l = []
    ip = []
    k=0

    pred = pred.lower()
    print(pred)
    for i in range(len(pred)):
        l.append(char2idx.get(pred[i]))
    #print(l)
    #print(char2idx.get("PAD"))        


    length = len(l)
    #print(length)

    if length < max_word_length :
        k = max_word_length - length
        #print(k)

        for i in range (0,k) :
            l.append(char2idx.get("PAD"))    

    else :
        l = l[:max_word_length]
    #ip.append(l)

    l = np.array([l])
    #print(l)
    
    return l

In [9]:
def pred_new_vec(l) :
    train_char_vectors = model.predict(l)

    #print(train_char_vectors)
    
    return train_char_vectors

In [10]:
def form_list(R) :
    lR = []
    for r in R :
        a = pred_new_num_enc(r)
        #print(a)
        #print("\n")
        a = pred_new_vec(a)
        #print(a)
        #print("\n")    
        lR.append(a)
        #print(lR)
        
    lR = np.array(lR)
    #print(lR.shape)

    #np.reshape(lR1,(5,10))
    lR = lR[:,0,:]

    #print(lR1)
    #print("\n")
    
    print(lR.shape)
    print("\n")
    #print(lR)
    print(lR.shape)
    lR = lR.flatten()

    print(lR.shape)
    
    lR = list(lR)
    
    print(lR)
    print(len(lR))
    
    return lR

In [11]:
lR1 = form_list(R1)

name
address
contact
sex
guardian's-name
(5, 10)


(5, 10)
(50,)
[0.0072833523, -0.0052619516, 0.0014931209, 0.022276362, 0.021285735, 0.009103995, 0.021347705, -0.0058705215, 0.011582155, 0.0008652805, 0.007286784, -0.005284902, 0.0016639757, 0.022247, 0.021231093, 0.009027433, 0.012712594, 0.009841611, 0.007956776, 0.0076008826, 0.007228851, -0.0053097815, 0.0015090611, 0.022216588, 0.021072358, 0.00032139357, 0.04641819, 0.02060448, 0.0068478878, 0.0026252328, 0.0073550274, -0.005236847, 0.001607551, 0.022356726, 0.021429999, 0.017235031, 0.003480987, -0.0031980716, 0.018067095, -0.0077027776, 0.006490308, -0.0054110517, 0.00026343076, 0.021396365, 0.018491117, -0.012052889, 0.017090514, -0.0060704397, 0.0064239656, -0.010192778]
50


In [12]:
lR2 = form_list(R2)

name
adres
phone-no
gender
father's-name
(5, 10)


(5, 10)
(50,)
[0.0072833523, -0.0052619516, 0.0014931209, 0.022276362, 0.021285735, 0.009103995, 0.021347705, -0.0058705215, 0.011582155, 0.0008652805, 0.007318842, -0.005256898, 0.0016047201, 0.022305153, 0.02133302, 0.010851118, 0.011658778, 0.0067960955, 0.007759862, 0.004300004, 0.0075333873, -0.0051642493, 0.0017032514, 0.022624344, 0.02176947, 0.013316419, 0.044364195, 0.019738289, 0.008953016, -0.012345662, 0.0073416815, -0.0052552256, 0.001600359, 0.022350442, 0.021388939, 0.007347314, 0.026630796, 0.0037201566, 0.0057451525, -0.0045844354, 0.0066208355, -0.005510031, 0.0006111294, 0.021552052, 0.019402089, -0.0068806475, 0.007715619, 0.0013025934, -0.0033829377, 0.0049925377]
50


In [13]:
inputs_records = lR1 + lR2

print(inputs_records)

print("\n")
print(len(inputs_records))

[0.0072833523, -0.0052619516, 0.0014931209, 0.022276362, 0.021285735, 0.009103995, 0.021347705, -0.0058705215, 0.011582155, 0.0008652805, 0.007286784, -0.005284902, 0.0016639757, 0.022247, 0.021231093, 0.009027433, 0.012712594, 0.009841611, 0.007956776, 0.0076008826, 0.007228851, -0.0053097815, 0.0015090611, 0.022216588, 0.021072358, 0.00032139357, 0.04641819, 0.02060448, 0.0068478878, 0.0026252328, 0.0073550274, -0.005236847, 0.001607551, 0.022356726, 0.021429999, 0.017235031, 0.003480987, -0.0031980716, 0.018067095, -0.0077027776, 0.006490308, -0.0054110517, 0.00026343076, 0.021396365, 0.018491117, -0.012052889, 0.017090514, -0.0060704397, 0.0064239656, -0.010192778, 0.0072833523, -0.0052619516, 0.0014931209, 0.022276362, 0.021285735, 0.009103995, 0.021347705, -0.0058705215, 0.011582155, 0.0008652805, 0.007318842, -0.005256898, 0.0016047201, 0.022305153, 0.02133302, 0.010851118, 0.011658778, 0.0067960955, 0.007759862, 0.004300004, 0.0075333873, -0.0051642493, 0.0017032514, 0.02262434

In [14]:
'''arr = []

for rec in inputs_records :
    arr.append(rec)
    
print(arr)'''

'arr = []\n\nfor rec in inputs_records :\n    arr.append(rec)\n    \nprint(arr)'

In [15]:
inputs_records = np.array([inputs_records])
print(inputs_records.shape)

model = Sequential()
model.add(Dense(50, input_shape = (100,), activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()

(1, 100)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 5,101
Trainable params: 5,101
Non-trainable params: 0
_________________________________________________________________


In [19]:
prediction = model.predict(inputs_records)
print(prediction)

[[0.4996919]]


In [24]:
print(round(prediction[0,0]))

0.0
